# AES

## Chiffrement et déchiffrement d'un message avec AES et une clé de 256 bits

In [11]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from os import urandom

def encrypt(key, message):
    """Chiffre un message avec AES et une clé de 256 bits.

    Args:
        key (bytes): Clé de 256 bits.
        message (bytes): Message à chiffrer.

    Returns:
        bytes: Message chiffré.
    """
    iv = urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message) + padder.finalize()
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return iv + ciphertext

def decrypt(key, ciphertext):
    """Déchiffre un message avec AES et une clé de 256 bits.

    Args:
        key (bytes): Clé de 256 bits.
        ciphertext (bytes): Message chiffré.

    Raises:
        ValueError: Le message n'est pas de la bonne taille, il doit être un multiple de 16 octets.

    Returns:
        str: Message déchiffré.
    """
    if len(ciphertext[16:]) % 16 != 0:
        raise ValueError(f'Le message ({len(ciphertext[16:])} octets) n\'est pas de la bonne taille, il doit être un multiple de 16 octets')
    cipher = Cipher(algorithms.AES(key), modes.CBC(ciphertext[:16]), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext[16:]) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    unpadded = unpadder.update(decrypted) + unpadder.finalize()
    return unpadded.decode()

key = "01" * 128
key = int(key, 2).to_bytes(len(key) // 8, byteorder='big')

message = b'Test de chiffrement AES avec CBC et un padding PKCS7'
ciphertext = encrypt(key, message)
print("Le message chiffré :", ciphertext)

decrypted = decrypt(key, ciphertext)
print("Le message déchiffré :", decrypted)

Le message chiffré : b'\x13\x1c\x08\x95{z\xa3,\xc5\x1fH\xebh\x05\x0f\x9f:\xe3q%"\xb8\xd5\n\xc1\xf9\x03\xec\x13_\xea\x03\x87W\xdb\x14\xf8\xcf\xa9\xe7\xeb\x9f\x1b\x83\xb5\x96Rj\xec\xa7\xc8{\x90\x11\x8f\x8f\x01\xfaA\x9c\xfaf^\xa2(\x92\x05\xc8R\x1f\xaa|\x02c~c\x88\xab\xcfd'
Le message déchiffré : Test de chiffrement AES avec CBC et un padding PKCS7


## Recherche de la clé dans les images

In [12]:
from PIL import Image

def dechiffrer_image(path):
    """Permet de récupérer la clé de chiffrement d'une image.

    Args:
        path (str): Chemin vers l'image.

    Returns:
        str: La clé de chiffrement.
    """
    image = Image.open(path)
    pixels = image.load()
    (largeur, hauteur) = image.size
    return ''.join([str(pixels[x, y] % 2) for y in range(hauteur) for x in range(largeur)])[:64]

key = dechiffrer_image('data/rossignol2.bmp')
print(f'La clé est en binaire : {key} (en hexadécimal : {hex(int(key, 2))})')

La clé est en binaire : 1110011101101101001100010011111110010010101110011001000001001100 (en hexadécimal : 0xe76d313f92b9904c)
